In [2]:
import os
import pickle
import time
from multiprocessing import Pool
from time import sleep

import networkx as nx
import numpy as np
# import pandana as pdna
import pandas as pd
from tqdm.notebook import trange, tqdm
import igraph as ig
from scripts import graph_osm_loader, utils, pipeline
from scripts.pfa import Path

Примеры как резолвить комьюнити

In [5]:
GRAPH_ID = 'R13470549'  # R13470549 R2555133 R337422

In [6]:
# примеры id есть в graph_osm_loader.py
g = graph_osm_loader.get_graph(GRAPH_ID)  # загрузка графа
len(g.nodes), len(g.edges)

(723, 1046)

In [7]:
node2id = {u: i for i, u in enumerate(g.nodes())}
nodes_x = [d['x'] for _, d in g.nodes(data=True)]
nodes_y = [d['y'] for _, d in g.nodes(data=True)]
edges_from = [node2id[u] for u, _ in g.edges()]
edges_to = [node2id[v] for _, v in g.edges()]
edges_w = [d['length'] for u, v, d in g.edges(data=True)]
df = pd.DataFrame.from_dict({
    'x': nodes_x,
    'y': nodes_y
})
df1 = pd.DataFrame.from_dict(
    {
        'f': edges_from,
        't': edges_to,
        'w': edges_w
    }
)


In [8]:
G = ig.Graph(n=len(nodes_x), edges=[[edges_from[i], edges_to[i]] for i in range(len(edges_to))],
                 edge_attrs={'weight': edges_w})

In [ ]:
net_ch = pdna.Network(df['x'], df['y'], df1['f'], df1['t'], df1[['w']])
# net_ch.precompute(10_000)

In [9]:
points= utils.generate_points(g)

In [10]:
from scripts import pfa


class IGraphPfa(pfa.PathFinding):
    def find_path(self, start: int, end: int)-> Path:
        p = G.get_shortest_paths(node2id[start],node2id[end], weights=G.es["weight"], output="epath",algorithm = 'dijkstra')
        l = 0
        for e in p[0]:
            l += G.es[e]["weight"]
        return l,p
    
class PandanaPfa(pfa.PathFinding):
    def find_path(self, start: int, end: int)-> Path:
        return net_ch.shortest_path_length(node2id[start], node2id[end])

In [11]:
stat = pipeline.PfaComparator(
    baseline=IGraphPfa(g = g),
    test_algorithm=PandanaPfa(g =g),
    points=points,
    workers=10
).compare()
stat

start 0


find paths:   0%|          | 0/100 [00:00<?, ?it/s]

start 1


find paths:   0%|          | 0/100 [00:00<?, ?it/s]

start 2


...

start 8


find paths:   0%|          | 0/100 [00:00<?, ?it/s]

start 9


find paths:   0%|          | 0/100 [00:00<?, ?it/s]


        err_max:    0.0000
        err_min:    0.0000
        err_median: 0.0000
        err_mean:   0.0000
        
        acceleration_max:    4.6941
        acceleration_min:    0.6387
        acceleration_median: 1.1120
        acceleration_mean:   1.1464
        